In [1]:
import ee
import geemap
import os

In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()

In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')

In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
region = ee.FeatureCollection('users/franciscoalbornoz/Aysen')
Map.addLayer(region, {}, 'Aysen')

In [8]:
start_date = '2020-12-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)


In [9]:
# l8testnubes = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
#     .filterBounds(roi) \
#     .filterDate(start_date, end_date)\
#     .map(Cloud2)

In [10]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(roi, 10)
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [12]:
# ndwi = .normalizedDifference(['B3','B5'])

In [13]:
def Cloud(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20))    

In [14]:
# def Cloud2(image):
#     scored = ee.Algorithms.Landsat.simpleCloudScore(image)
#     return image.updateMask(scored.select(['cloud']).lt(10))

In [15]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [16]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [17]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [18]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [19]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [20]:
def aguamask(image):
    return image.updateMask(ndwi.gte(0.4))    

In [21]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [22]:
composicion = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY).map(addNDWI).map(addNDGI)

In [23]:
# coleccion = l8testnubes.map(addNDVI).map(addDate).map(addMonth).map(addDOY).map(addNDWI).map(addNDGI)

In [24]:
# mascarasnubes = coleccion.qualityMosaic('NDVI')

In [25]:
greenestregion = composicion.qualityMosaic('NDVI').clip(region)
greenest = composicion.qualityMosaic('NDVI')

In [26]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy',
 'NDWI',
 'NDGI']

In [27]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [28]:
ndwi = greenest.select('NDWI')


In [29]:
mascaraagua = composicion.map(aguamask)

In [30]:
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndwi, {'palette': palette}, 'NDWI')
# ndwirgb = mascaraagua.visualize({
#   'min': 0.5,
#   'max': 1,
#   'palette': ['00FFFF', '0000FF']
# }); 

In [31]:
otroproceso = mascaraagua.map(Cloud)
greenest3 = otroproceso.qualityMosaic('NDVI')

In [32]:
ndgi = greenest.select('NDGI')
palette = ['#D0D3D4', '#979A9A', '#7B7D7D', '#76D7C4', '#E74C3C', '#1ABC9C','#6E2C00', '#76D7C4','#48C9B0', '#17A589', '#148F77', '#F7DC6F', '#F4D03F']
Map.addLayer(ndgi, {'palette': palette}, 'NDGI')

In [33]:
Map.addLayer(greenest, visParams, 'Greenest pixel')

In [34]:
# Map.addLayer(mascarasnubes, visParams, 'mascara')

In [35]:
Map.addLayer(greenest3, visParams, '3')

In [36]:
# greenest3rgb = mascaraagua.visualize(visParams)

In [37]:
testmosaico = ee.ImageCollection([greenest3, ndwi]).mosaic()
Map.addLayer(testmosaico, {}, 'mosaico')

In [38]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(bottom=160691.0, center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['positi…

In [39]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [40]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [41]:
landcover_2018.bandNames().getInfo()

['b1']

In [42]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [43]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [44]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [45]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [46]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [47]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [48]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [49]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')


In [50]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [51]:
dataset = greenest3.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [52]:
datasetval = greenest3.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [53]:
dataset.size().getInfo()

0

In [54]:
datasetval.size().getInfo()

0

In [55]:
print(dataset.first().getInfo())

None


In [56]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'NDWI',
 'NDGI']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [57]:
# print(training.first().getInfo())

In [58]:
# print(validation.first().getInfo())

In [59]:
randomforest = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)
resultRF = greenest.select(bands).classify(randomforest)

In [60]:
SVM = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )
trained = SVM.train(dataset, label, bands)
classifiedSVM = greenest.classify(trained)

In [61]:
smc = ee.Classifier.smileCart().train(dataset, label, bands)
classifiedsmc = greenest.select(bands).classify(smc)

In [62]:
# Clasificando con las mismas bandas de entrenamiento
# result1 = greenest.select(bands).classify(randomforest)
# result2 = greenest.select(bands).classify(SVM)
# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [63]:
# result.getInfo()

In [64]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [65]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [66]:
landcover_randomforest = resultRF.set('classification_class_values', class_values)
landcover_randomforest = landcover_randomforest.set('classification_class_palette', class_palette)

In [67]:
Map.addLayer(landcover_randomforest, {}, 'Land cover Random Forest Región')

EEException: No valid training data were found.

In [ ]:
landcover_svm = classifiedSVM.set('classification_class_values', class_values)
landcover_svm = landcover_svm.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover_svm, {}, 'Land cover SVM Región')

In [ ]:
landcover_cart = classifiedsmc.set('classification_class_values', class_values)
landcover_cart = landcover_cart.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover_cart, {}, 'Land cover SCART Región')

In [ ]:
Map

RF

In [ ]:
train_accuracy = randomforest.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validacion = datasetval.classify(randomforest)

In [ ]:
validacion.first().getInfo()

In [ ]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [ ]:
testAccuracy.getInfo()

In [ ]:
testAccuracy.accuracy().getInfo()

In [ ]:
testAccuracy.kappa().getInfo()

In [ ]:
testAccuracy.producersAccuracy().getInfo()

In [ ]:
testAccuracy.consumersAccuracy().getInfo()

SVM

In [ ]:
train_accuracy1 = trained.confusionMatrix()

In [ ]:
train_accuracy1.getInfo()

In [ ]:
train_accuracy1.accuracy().getInfo()

In [ ]:
train_accuracy1.kappa().getInfo()

In [ ]:
train_accuracy1.producersAccuracy().getInfo()

In [ ]:
train_accuracy1.consumersAccuracy().getInfo()

In [ ]:
validacion1 = datasetval.classify(trained)

In [ ]:
testAccuracy1 = validacion1.errorMatrix('ID', 'classification')

In [ ]:
testAccuracy1.getInfo()

In [ ]:
testAccuracy1.accuracy().getInfo()

In [ ]:
testAccuracy1.kappa().getInfo()

In [ ]:
testAccuracy1.producersAccuracy().getInfo()

In [ ]:
testAccuracy1.consumersAccuracy().getInfo()

SCART

In [ ]:
train_accuracy = smc.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = datasetval.classify(smc)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()